In [1]:
import pandas as pd
import numpy as np
import json
from ast import literal_eval
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
#brings in the dfs, cleans them

df1 = pd.read_csv("tmdb_5000_movies.csv")
df1['release_date'] = pd.to_datetime(df1['release_date']).apply(lambda x: x.date())
json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
for column in json_columns:
    df1[column] = df1[column].apply(json.loads)
df1 = df1[["budget", "genres", "original_title", "popularity", "production_companies",
                      "release_date", "revenue", "runtime", "vote_average", "vote_count", "overview"]]
df1 = df1.reset_index(drop=True)

for index, row in df1.iterrows():
    g=[]
    for i in range(len(df1["genres"][index])):
        g.append(df1["genres"][index][i].get("name"))
    df1.at[index, 'genres'] =g
           
for index, row in df1.iterrows():
    c=[]
    for i in range(len(df1["production_companies"][index])):
        c.append(df1["production_companies"][index][i].get("name"))
    df1.at[index, 'production_companies'] =c
    
df2 = pd.read_csv("2017_movie_info.csv")
df2['release_date'] = pd.to_datetime(df2['release_date']).apply(lambda x: x.date())
df2["genres"] = df2["genres"].apply(literal_eval)
df2["production_companies"] = df2["production_companies"].apply(literal_eval)
df2.drop('id', axis=1, inplace=True)
df2 = df2.reset_index(drop=True)

df = pd.concat([df1,df2],axis=0, join='outer',ignore_index=True)

#clean_df = df.loc[df["budget"] != 0]
#clean_df = df.loc[df["revenue"] != 0]
clean_df = df.loc[df["vote_count"]>100]
#clean_df = df.loc[df["production_companies"] != []]
#clean_df = df.loc[df["genres"] != []]

len(clean_df)

3490

In [4]:
#runs vader
compound_score =[]
for index, row in clean_df.iterrows():
    try:
        results = analyzer.polarity_scores(clean_df["overview"][index])
        comp = results["compound"]  
    except:
        comp=None
    compound_score.append(comp)
        
clean_df["compound_score"] = compound_score
clean_df.to_csv("movies_with_compound.csv", index=False, header=True)

/anaconda3/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [11]:
print(clean_df["budget"][0]*len(df["genres"][0]))

948000000


In [12]:
clean_df.head(2)

,budget,genres,original_title,popularity,production_companies,release_date,revenue,runtime,vote_average,vote_count,overview,compound_score
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",Avatar,150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",2009-12-10,2787965087,162.0,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",-0.3612
1,300000000,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",2007-05-19,961000000,169.0,6.9,4500,"Captain Barbossa, long believed to be dead, ha...",-0.3919


In [9]:
genres_list = []
budget_list = []
title_list = []
popularity_list = []
company_list = []
revenue_list = []
runtime_list = []
rating_list = []
compound_list = []
for i in range(len(clean_df)):
    for item in clean_df["genres"][i]:
        genres_list.append(item)
        budget_list.append(clean_df["budget"][i])
        title_list.append(clean_df["original_title"][i]
        popularity_list.append(clean_df["popularity"][i]*len(df["genres"][i]))
        company_list.append(clean_df["production_companies"][i]*len(df["genres"][i]))
        revenue_list.append(clean_df["revenue"][i]*len(df["genres"][i]))
        runtime_list.append(clean_df["runtime"][i]*len(df["genres"][i]))
        rating_list.append(clean_df["vote_average"][i]*len(df["genres"][i]))
        compound_list.append(clean_df["compound_score"][i]*len(df["genres"][i]))

print(len(genres_list))  
print(len(budget_list))
print(len(title_list))

KeyError: 83

In [ ]:
#Unique genres list

unique_genres = []

for row in clean_df["genres"]:
    if clean_df["genres"]["name"] not in unique_genres:
            unique_genres.append(clean_df["genres"]['name'])
unique_genres            

In [ ]:
##THIS BLOCK IS NOT BEING USED

#after single code is complete, create a loop for each genre and a master blank list to append each genre name and avg value to
#create a df with that list to use to plot the data
import statistics
count = 0
Action = []
checks = []
for index,row in clean_df1.iterrows():
    #print(row)
    
    lista = []
    jdata = json.loads(clean_df1["genres"][index])
#     Action.append(jdata)
#     print("Hi")
#     print(len(Action))
#     break
    for d in jdata:
#         if d["name"] == "Action":
#             act = clean_df1["vote_average"]
        if d["name"] not in lista:
            lista.append(d['name'])
#         Action.append(act)
#         break    
            
#             Action.append(d['name'])
#     clean_df1.set_value(index, "genres", lista)        
#             count += 1
#     print(lista)
# Action_df = clean_df1.loc[clean_df1["genres"] == "Action"]
    if "Action" in lista:
#         Action.append(clean_df1["vote_average"]) #use a loc, this line is the issue
        check= clean_df1["revenue"]
        act = clean_df1["vote_average"]
        name1 = "Action"
#         print(act)
#         print(name1)
        Action.append({"genre":name1,"Voter Avg":act, "Revenue":check})
# #     check= clean_df1.loc[clean_df1["genres"]]        
        break
        
        
# df.set_value('index pos', 'column', value to put in)        
        
# ActionAvg = statistics.mean(Action)
# Action2 = pd.DataFrame.from_dict(Action)
# Action = Action.transpose()
print(Action)
# ActionAvg = Action.mean(axis=1)
# checks.append(ActionAvg)
# checks
# clean_df1.head()
# print(lista)

In [ ]:
#genres code
import statistics

Action = []
Adventure = []
Fantasy = []
Science_Fiction = []
Crime = []
Drama = []
Thriller = []
Animation = []
Family = []
Western = []
Comedy = []
Romance = []
Horror = []
Mystery = []
History = []
War = []
Music = []
Documentary = []
TV_Movie = []



for index,row in clean_df1.iterrows():
    
    jdata = json.loads(clean_df1["genres"][index])

    for d in jdata:

        if d["id"] == 28:
            Action.append(row)
            
        if d["id"] == 12:
            Adventure.append(row) 
        
        if d["name"] == "Fantasy":
            Fantasy.append(row)  
            
        if d["name"] == "Science Fiction":
            Science_Fiction.append(row)  
            
        if d["name"] == "Crime":
            Crime.append(row)  
            
        if d["name"] == "Drama":
            Drama.append(row)  
            
        if d["name"] == "Thriller":
            Thriller.append(row)  
            
        if d["name"] == "Animation":
            Animation.append(row)  
        
        if d["name"] == "Family":
            Family.append(row)  
        
        if d["name"] == "Western":
            Western.append(row)  
            
        if d["name"] == "Comedy":
            Comedy.append(row)  
            
        if d["name"] == "Romance":
            Romance.append(row)  
            
        if d["name"] == "Horror":
            Horror.append(row)  
         
        if d["name"] == "Mystery":
            Mystery.append(row)  
            
        if d["name"] == "History":
            History.append(row)  
            
        if d["name"] == "War":
            War.append(row)  
            
        if d["name"] == "Music":
            Music.append(row)  
            
        if d["name"] == "Documentary":
            Documentary.append(row)  
            
        if d["name"] == "TV Movie":
            TV_Movie.append(row)      
        

Action_df = pd.DataFrame(Action)
Adventure_df = pd.DataFrame(Adventure)
Fantasy_df = pd.DataFrame(Fantasy)
Science_Fiction_df = pd.DataFrame(Science_Fiction)
Crime_df = pd.DataFrame(Crime)
Drama_df = pd.DataFrame(Drama)
Thriller_df = pd.DataFrame(Thriller)
Animation_df = pd.DataFrame(Animation)
Family_df = pd.DataFrame(Family)
Western_df = pd.DataFrame(Western)
Comedy_df = pd.DataFrame(Comedy)
Romance_df = pd.DataFrame(Romance)
Horror_df = pd.DataFrame(Horror)
Mystery_df = pd.DataFrame(Mystery)
History_df = pd.DataFrame(History)
War_df = pd.DataFrame(War)
Music_df = pd.DataFrame(Music)
Documentary_df = pd.DataFrame(Documentary)
TV_Movie_df = pd.DataFrame(TV_Movie)


##The code under here works to be used elsewhere
# sums = []
# for index,row in Adventure_df.iterrows():
#     sums.append(Adventure_df["vote_average"][index])
# averages = (sum(sums)/len(sums)).round(2)    
# averages
##The code above works to be used elsewhere

In [ ]:
#company code NOT FINISHED
unique_companies = []
unique_companies_to_list=[]
for row in clean_df1["production_companies"]:
    jdata = json.loads(row)
    for d in jdata:
        if d["id"] not in unique_companies:
            unique_companies.append(d['id'])
            unique_companies_to_list.append(d['id'])
for n in unique_companies_to_list:
    globals()[n] = []    

In [ ]:
#used to create empty lists out of each element in a list
# yup = ['a', 'b', 'c']
# for n in yup:
#     globals()[n] = []
unique_companies_to_list


In [ ]:
#example code NOT USED
l={1:"one",2:"two"}
print(l)
for k,v in l.items():
    print(k)
    

In [ ]:
#2017 names
test_text = pd.read_csv('Names_2017.csv')
test_text = pd.DataFrame(test_text)
test_text.head()

In [ ]:
for movie in test_text["Title"]:
    print(movie)

In [ ]:
#zip genre and codes
unique_genres = []
g_codes = []
for row in clean_df1["genres"]:
    jdata = json.loads(row)
    for d in jdata:
        if d["name"] not in unique_genres:
            unique_genres.append(d['name'])
            g_codes.append(d['id'])
zipped = zip(g_codes, unique_genres)  
list(zipped)

In [ ]:
#vader code was written here and copy/pasted to the top
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# analyzer = SentimentIntensityAnalyzer()
# compund = []

# for index, row in clean_df1.iterrows():
#     results = analyzer.polarity_scores(clean_df1["overview"][index])

#     comp = results["compound"]
#     compund.append(comp)
# clean_df1["compound_score"] = compund   
# clean_df1.head()